# Time series forecasting

Time series forecasting is one of the most common problems one encounters in the machine learning domain. You can find time series forecasting problems everywhere, whether its predicting the stock market prices or predicting sales for a particular item on an ecommerce website or predicting readings of an IoT device placed in a factory. Timeseries forecasting has be approached using quite a few methods. You can find a brief of the different approaches here.

We will be using Amazon SageMaker's builtin DeepAR algorithm to perform this task. DeepAR is an algorithm that uses Recurrent Neural Networks to forecast univariate time series. There are various aspects of DeepAR that are unique and you can find out more about how DeepAR differs from other algorithms here.

The dataset we are using for this is [this one](https://archive.ics.uci.edu/ml/datasets/Individual+household+electric+power+consumption). The dataset has 2 million readings of household electricity consumption in Sceaux between 2006 and 2010. We intend to train a model that will effectively and accurately predict the electricity consumption in various rooms within the household.

## Setup

_This notebook was created and tested on an ml.m4.2xlarge_
- You will require an Amazon S3 bucket to store the data in. This has to be in the same region as your Amazon SageMaker instance. [Here](https://docs.aws.amazon.com/AmazonS3/latest/user-guide/create-bucket.html) are the details on how you can create an Amazon S3 bucket.
- An IAM role ARN to provide training and hosting access to your data. You can find further details in the documentation on how to create these roles [here](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html).

## Data

You will need to download the data from the UCI website. [Here](https://archive.ics.uci.edu/ml/datasets/Individual+household+electric+power+consumption) are the details of the dataset.
You can either download this data to the notebook instance or to an Amazon S3 bucket of your choice.

In [ ]:
from zipfile import ZipFile
from io import BytesIO
from dateutil.parser import parse
from sagemaker import get_execution_role,Session,estimator,predictor ## install this with 'sudo pip install sagemaker'

roleARN = get_execution_role()
bucket = <your-s3-bucket-name> ## Replace with your bucket name
prefix = 'sagemaker/data/Household_Electricity_Consumption' ## Replace with the folder structure inside your bucket or simply ''

In [ ]:
import boto3
import datetime
import json
import bisect
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time
import sagemaker.amazon.common as smac

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00235/household_power_consumption.zip

Lets extract the data.

In [ ]:
with ZipFile('household_power_consumption.zip') as zfl:
    zfl.extractall()
    print("Following files have been extracted: \n{}".format(zfl.namelist()))

Lets take a first look at the data. Notice the number of data fields, there is also a relationship between the fields, you can examine the relationship as explained on the webssite where the dataset is hosted. Also, observe that the scales for the fields are very different, so, for example, observe that Global_active_power and Sub_metering_1 have very different scales. Noice also that we haven't normalized the data yet.

In [ ]:
!head household_power_consumption.txt

In [ ]:
!wc -l household_power_consumption.txt

Lets create the dataframe and examine it.

In [ ]:
inputdata = pd.read_csv('./household_power_consumption.txt',sep=';',header=0,index_col=[0],parse_dates=[0,1],dayfirst=True)

In [ ]:
inputdata.head()

We are doing quite a few things here. Notice from the previous output when examining the dataframe we just created above, the data has a granularity of 60 seconds. While we can use this for time series forecasting, we are not able to consider quite a few factors that we would be able to if the data had a larger frequency. With this in mind we are changing the data sampling from a minute to an hour. We are also replacing missing values with NaNs to avoid bias, you can do this with DeepAR, you can read all about DeepARs support for missing values [here](https://aws.amazon.com/blogs/machine-learning/amazon-sagemaker-deepar-now-supports-missing-values-categorical-and-time-series-features-and-generalized-frequencies/). 

We are also creating a few new columns that we have added to the dataframe to ease our calculations.

In [ ]:
inputdata.replace('?',np.nan,inplace=True)
inputdata[['Global_active_power','Global_reactive_power','Voltage','Global_intensity','Sub_metering_1','Sub_metering_2','Sub_metering_3']] = inputdata[['Global_active_power','Global_reactive_power','Voltage','Global_intensity','Sub_metering_1','Sub_metering_2','Sub_metering_3']].apply(pd.to_numeric)
inputdata['Date2'] = inputdata.index.values
inputdata['TimeHrs'] = inputdata['Time'].dt.hour
inp = inputdata
grp = inp.groupby(['Date2','TimeHrs'],as_index=False)
inp1 = grp['Global_active_power','Global_reactive_power','Voltage','Global_intensity','Sub_metering_1','Sub_metering_2','Sub_metering_3'].agg(np.mean)

In [ ]:
inp1.head(48)

DeepAR supports Dynamic Features, these are crucial in training a model that considers factors that may affect our predictions but are outside of the data that we have. In this case, we are considering solstices within a year to make predictions in terms of how the weather might be affecting electricity consumption. For example, sub-meters serving certain appliances, like air conditioning will observe particularly high consumption in the hot summer months i.e. if the date ranges we are predicting for are closer to the summer solstice. We used the [pyEphem package](https://pypi.org/project/pyephem/) to build an array of yearwise (_for the years under consideration_) dates for solstices (_as they can differe slightly from year to year_).

Since this is a one time exercise that need not be repeated everytime this notebook is created, we have added the list of solstice dates as an array below and have also added the code that you can use to generate this array.

In [ ]:
sollistv2 = [datetime.date(2000, 6, 21), datetime.date(2000, 12, 21), datetime.date(2001, 6, 21), datetime.date(2001, 12, 21), datetime.date(2002, 6, 21), datetime.date(2002, 12, 22), datetime.date(2003, 6, 21), datetime.date(2003, 12, 22), datetime.date(2004, 6, 21), datetime.date(2004, 12, 21), datetime.date(2005, 6, 21), datetime.date(2005, 12, 21), datetime.date(2006, 6, 21), datetime.date(2006, 12, 22), datetime.date(2007, 6, 21), datetime.date(2007, 12, 22), datetime.date(2008, 6, 20), datetime.date(2008, 12, 21), datetime.date(2009, 6, 21), datetime.date(2009, 12, 21), datetime.date(2010, 6, 21), datetime.date(2010, 12, 21), datetime.date(2011, 6, 21), datetime.date(2011, 12, 22), datetime.date(2012, 6, 20), datetime.date(2012, 12, 21), datetime.date(2013, 6, 21), datetime.date(2013, 12, 21), datetime.date(2014, 6, 21), datetime.date(2014, 12, 21), datetime.date(2015, 6, 21), datetime.date(2015, 12, 22), datetime.date(2016, 6, 20), datetime.date(2016, 12, 21), datetime.date(2017, 6, 21), datetime.date(2017, 12, 21), datetime.date(2018, 6, 21), datetime.date(2018, 12, 21)]

You can build the _sollistv2_ array like its done here below, you could use this code in a file in the terminal on this Jupyter Notebook instance. You can then use the array for the code in the next cell. You would need to install pyephem using pip prior to running this program. Although for the purposes for this notebook, you don't have to run this again as the data output by this code will remain the same and has been assigned to the _sollistv2_ above.

```python
#!/usr/bin/env python

import ephem
import datetime

def main():
        sollist,sollistv2 = [],[]
        for yr in range(2000,2019):
                nsol1 = ephem.next_solstice(str(yr))
                nsol2 = ephem.next_solstice(nsol1)
                strdt = nsol1.datetime().strftime("%Y-%m-%d")
                sollist.append(strdt)
                sollistv2.append(datetime.datetime.strptime(sollist[-1],"%Y-%m-%d").date())
                strdt = nsol2.datetime().strftime("%Y-%m-%d")
                sollist.append(strdt)
                sollistv2.append(datetime.datetime.strptime(sollist[-1],"%Y-%m-%d").date())
        print(sollistv2)

if __name__=="__main__":
        main()
```

## Generate Dynamic Features

Dynamic Features are optional input that you can provide as part of the training input to DeepAR. They have be an array of arrays of floats or integers that represent custom features. In this case, we are using the distance from the previous and next solstices for a given date. These, as described above, are going to be our custom features.

We are populating values for these dynamic features. Note that while DeepAR can handle missing values in data series, dynamic features cannot have NaN/missing values.

In [ ]:
## We calculate the distances from solstices and return the values, the input is observation date 
## i.e. same name as the column in our dataframes(inputs). We then populate the other columns.
def getDistFromSolstices(obdt):
    indx = bisect.bisect(sollistv2,obdt.date())
    return (obdt.date() - sollistv2[indx-1]).days,(sollistv2[indx] - obdt.date()).days

def populateDistance(data):
    dsfromlast,dsfromnext = [],[]
    for row in data.itertuples():
        dfromlast,dfromnext = getDistFromSolstices(row[1]) #Get 'Date2' column
        dsfromlast.append(dfromlast)
        dsfromnext.append(dfromnext)
    data = data.assign(distanceFromLastSolstice=pd.Series(dsfromlast,index=data.index))
    data = data.assign(distanceFromNextSolstice=pd.Series(dsfromnext,index=data.index))
    return data

In [ ]:
processedDF = populateDistance(inp1)

In [ ]:
processedDF.head(48)

Fix all NaN values before we start writing this to a file.

In [ ]:
processedDF.fillna(value="NaN",inplace=True)

## Using Categories

Categorical features can be used to encode groups to which the record belongs. Here we group Global_active_power, Sub_metering_1, Sub_metering_2 and Sub_meter_3. We have created separate groups for Global_reactive_power and Voltage. The reason we use this kind of grouping is because the first group mentioned before have a relationship, ref. [dataset document here](https://archive.ics.uci.edu/ml/datasets/individual+household+electric+power+consumption).

## Separating training and test data

We will be separating our test and training data, using the data from some of the years that we resampled previous. Note the categories and dynamic features. 

In [ ]:
## Building the training data 

trdata = []

## Category for Global Active Power, Sub-Meter-1,2,3 -> 0
starttime = str(processedDF.iloc[0]['Date2'])[:10] +' '+str(processedDF.iloc[0]['TimeHrs'])+':00:00'
target = processedDF[processedDF['Date2'] < pd.to_datetime('2008-01').date()]['Global_active_power'].tolist()
cat = [0,0]
dynfeat = [processedDF[processedDF['Date2'] < pd.to_datetime('2008-01').date()]['distanceFromLastSolstice'].tolist(),processedDF[processedDF['Date2'] < pd.to_datetime('2008-01').date()]['distanceFromNextSolstice'].tolist()]
trdata.append({"start": starttime,
    "target": target,
    "cat": cat,
    "dynamic_feat": dynfeat})

## llly, for sub-meter-1
target = processedDF[processedDF['Date2'] < pd.to_datetime('2008-01').date()]['Sub_metering_1'].tolist()
cat = [0,1]
trdata.append({"start": starttime,
    "target": target,
    "cat": cat,
    "dynamic_feat": dynfeat})

## llly, for sub-meter-2
target = processedDF[processedDF['Date2'] < pd.to_datetime('2008-01').date()]['Sub_metering_2'].tolist()
cat = [0,2]
trdata.append({"start": starttime,
    "target": target,
    "cat": cat,
    "dynamic_feat": dynfeat})

## llly, for sub-meter-2
target = processedDF[processedDF['Date2'] < pd.to_datetime('2008-01').date()]['Sub_metering_3'].tolist()
cat = [0,3]
trdata.append({"start": starttime,
    "target": target,
    "cat": cat,
    "dynamic_feat": dynfeat})

target = processedDF[processedDF['Date2'] < pd.to_datetime('2008-01').date()]['Global_reactive_power'].tolist()
cat = [1,4]
trdata.append({"start": starttime,
    "target": target,
    "cat": cat,
    "dynamic_feat": dynfeat})

target = processedDF[processedDF['Date2'] < pd.to_datetime('2008-01').date()]['Voltage'].tolist()
cat = [2,5]
trdata.append({"start": starttime,
    "target": target,
    "cat": cat,
    "dynamic_feat": dynfeat})

random.shuffle(trdata)

f = open("trainingdata.json",'wb')
for datapoint in trdata:
    f.write(json.dumps(datapoint).encode("utf-8"))
    f.write("\n".encode('utf-8'))
f.close()

In [ ]:
## Building the test data

tedata = []

## Category for Global Active Power, Sub-Meter-1,2,3 -> 0

starttime = str(processedDF[(processedDF['Date2'] == '2008-01')].iloc[0]['Date2'])[:10]+' '+ '0' + str(processedDF[processedDF['Date2'] == '2008-01'].iloc[0]['TimeHrs'])+':00:00'

target = processedDF[(processedDF['Date2'] >= '2008-01') & (processedDF['Date2'] < '2008-07')]['Global_active_power'].tolist()
cat = [0,0]
dynfeat = [processedDF[(processedDF['Date2'] >= '2008-01') & (processedDF['Date2'] < '2008-07')]['distanceFromLastSolstice'].tolist(),processedDF[(processedDF['Date2'] >= '2008-01') & (processedDF['Date2'] < '2008-07')]['distanceFromNextSolstice'].tolist()]
tedata.append({
    "start": starttime,
    "target": target,
    "cat": cat,
    "dynamic_feat": dynfeat
})

## llly, for sub-meter-1
target = processedDF[(processedDF['Date2'] >= '2008-01') & (processedDF['Date2'] < '2008-07')]['Sub_metering_1'].tolist()
cat = [0,1]
tedata.append({
    "start": starttime,
    "target": target,
    "cat": cat,
    "dynamic_feat": dynfeat
})

## llly, for sub-meter-2
target = processedDF[(processedDF['Date2'] >= '2008-01') & (processedDF['Date2'] < '2008-07')]['Sub_metering_2'].tolist()
cat = [0,2]
tedata.append({
    "start": starttime,
    "target": target,
    "cat": cat,
    "dynamic_feat": dynfeat
})

## llly, for sub-meter-2
target = processedDF[(processedDF['Date2'] >= '2008-01') & (processedDF['Date2'] < '2008-07')]['Sub_metering_3'].tolist()
cat = [0,3]
tedata.append({
    "start": starttime,
    "target": target,
    "cat": cat,
    "dynamic_feat": dynfeat
})

target = processedDF[(processedDF['Date2'] >= '2008-01') & (processedDF['Date2'] < '2008-07')]['Global_reactive_power'].tolist()
cat = [1,4]
tedata.append({
    "start": starttime,
    "target": target,
    "cat": cat,
    "dynamic_feat": dynfeat
})

target = processedDF[(processedDF['Date2'] >= '2008-01') & (processedDF['Date2'] < '2008-07')]['Voltage'].tolist()
cat = [2,5]
tedata.append({
    "start": starttime,
    "target": target,
    "cat": cat,
    "dynamic_feat": dynfeat
})

f = open("testdata.json",'wb')
for datapoint in tedata:
    f.write(json.dumps(datapoint).encode("utf-8"))
    f.write("\n".encode('utf-8'))
f.close()

Upload the training and testing files we created to our S3 location.

In [ ]:
training_key = 'trainingdata.json'
testing_key = 'testdata.json'
train_prefix   = '{}/{}'.format(prefix, 'train')
test_prefix = '{}/{}'.format(prefix,'test')

## Lets create the sagemaker session and upload the data from where Amazon SageMaker will pick it 
## up to put in the container
sg_sess = Session()
training_path  = sg_sess.upload_data(training_key, bucket=bucket, key_prefix=train_prefix)
testing_path = sg_sess.upload_data(testing_key, bucket=bucket, key_prefix=test_prefix)

## Prepare for the training

* Get the DeepAR image,
* Set the context length,
* Set the prediction length,
* Set the hyperparameters for the training job,
* Launch the training job

Finally, attach an estimator to the training job to check its status.

In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri

img = get_image_uri(boto3.Session().region_name,'forecasting-deepar')
prediction_length = 24 # hours ahead
context_length = 216 # hours prior

In [ ]:
from time import gmtime, strftime

job_name = 'DevDay-DeepARTraining-' + strftime("%Y-%m-%d-%H-%M-%S",gmtime())
print(job_name)
                                
## We choose a different output location to keep these separate from our old artifacts
output_location = 's3://{}/{}/output'.format(bucket,prefix)
print('training artifacts will be uploaded to: {}'.format(output_location))

create_training_params = \
{
    "AlgorithmSpecification": {
        "TrainingImage": img,
        "TrainingInputMode": "File"
    },
    "RoleArn": roleARN,
    "OutputDataConfig": {
        "S3OutputPath": output_location
    },
    "ResourceConfig": {
        "InstanceCount": 4,
        "InstanceType": "ml.c4.8xlarge",
        "VolumeSizeInGB": 10
    },
    "TrainingJobName": job_name,
    "HyperParameters": {
        "time_freq": 'H', # hourly series
        "context_length": str(context_length),
        "prediction_length": str(prediction_length), # number of data points to predict
        "num_dynamic_feat": "auto",
        "num_cells": "60", 
        "num_layers": "3",
        "likelihood": "gaussian", #real world data
        "epochs": "250", ## keep small, to save on time!!! 250 seems to give good results though
        "mini_batch_size": "512", ## Keep large, 128 gives good results
        "learning_rate": "0.01",
        "dropout_rate": "0.1"
    },
    "StoppingCondition": {
        "MaxRuntimeInSeconds": 60 * 240 # Give it four hours at best, could increase this for production scale
    },
    "InputDataConfig": [
        {
            "ChannelName": "train",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": training_path,
                    "S3DataDistributionType": "FullyReplicated"
                }
            },
            "CompressionType": "None",
            "RecordWrapperType": "None"
        },
        {
            "ChannelName": "test",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": testing_path,
                    "S3DataDistributionType": "FullyReplicated"
                }
            },
            "CompressionType": "None",
            "RecordWrapperType": "None"
        }
    ]
}

sgmaker = boto3.client('sagemaker')
sgmaker.create_training_job(**create_training_params)

## Lets check the status of the job to see if its complete, and lets wait until its done
status = sgmaker.describe_training_job(TrainingJobName=job_name)['TrainingJobStatus']
print(status)

In [ ]:
import sagemaker

tor = sagemaker.estimator.Estimator.attach(job_name)

## Model Deployment

We have created the model artifacts after training and stored them in the S3 location we specified when we created the training job. The steps to prepare for deployment are,
* Create a model,
* Create an endpoint configuration,
* Create an endpoint

In [ ]:
model_name = job_name

info = sgmaker.describe_training_job(TrainingJobName=job_name)
modeldata = info['ModelArtifacts']['S3ModelArtifacts']

container = {
    'Image': img,
    'ModelDataUrl': modeldata
}

created_model = sgmaker.create_model(
    ModelName = model_name,
    ExecutionRoleArn = roleARN,
    PrimaryContainer = container)

print(created_model['ModelArn'])

In [ ]:
endpoint_config_name = 'DevDayEndpointConfig-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_config_name)
created_endpoint_config = sgmaker.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType':'ml.m4.xlarge',
        'InitialInstanceCount':1,
        'ModelName':model_name,
        'VariantName':'AllTraffic'}])

print("Endpoint Config Arn: " + created_endpoint_config['EndpointConfigArn'])

In [ ]:
endpoint_name = 'DevDayEndpoint-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_name)
created_endpoint = sgmaker.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print(created_endpoint['EndpointArn'])

resp = sgmaker.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Status: " + status)

try:
    sgmaker.get_waiter('endpoint_in_service').wait(EndpointName=endpoint_name)
finally:
    resp = sgmaker.describe_endpoint(EndpointName=endpoint_name)
    status = resp['EndpointStatus']
    print("Arn: " + resp['EndpointArn'])
    print("Create endpoint ended with status: " + status)

    if status != 'InService':
        message = sgmaker.describe_endpoint(EndpointName=endpoint_name)['FailureReason']
        print('Training failed with the following error: {}'.format(message))
        raise Exception('Endpoint creation did not succeed')

## Inference and Visualization

Remember, that when querying the model we just trained for inference, we trained the model on multiple timeseries, and because the model has been trained on multiple timeseries we should be able to infer for more than one timeseries basically all of the timeseries we used for training. We will see below that we can use the same model to forecast multiple time series.

But first, lets write some convenience functions, the first of these builds the JSON formatted request that is required by DeepAR. The next invokes the endpoint and receives and returns the response from the model we deployed. Finally, we plot the results to compare what we predicted and the ground truth, just to see how well or badly we did.

In [ ]:
## A few convenience functions

def build_request(seriesstr):
    '''This function builds a request for a specific time series'''
    if processedDF[processedDF['Date2'] == '2008-01'].iloc[0]['TimeHrs'] < 10:
        starttime = str(processedDF[(processedDF['Date2'] == '2008-08')].iloc[0]['Date2'])[:10]+' '+ '0' + str(processedDF[processedDF['Date2'] == '2008-08'].iloc[0]['TimeHrs'])+':00:00'
    else:
        starttime = str(processedDF[(processedDF['Date2'] == '2008-08')].iloc[0]['Date2'])[:10]+' '+ str(processedDF[processedDF['Date2'] == '2008-08'].iloc[0]['TimeHrs'])+':00:00'

    target = processedDF[(processedDF['Date2'] >= '2008-08-01') & (processedDF['Date2'] < '2008-08-14')][seriesstr].tolist()
    cat = [0,0]
    dynfeat = [processedDF[(processedDF['Date2'] >= '2008-08-01') & (processedDF['Date2'] < '2008-08-15')]['distanceFromLastSolstice'].tolist(),processedDF[(processedDF['Date2'] >= '2008-08-01') & (processedDF['Date2'] < '2008-08-15')]['distanceFromNextSolstice'].tolist()]
    instances = []

    instances.append({
        "start": starttime,
        "target": target,
        "cat": cat,
        "dynamic_feat": dynfeat
    })

    request = {}
    request['instances'] = instances
    request['configuration'] = {}
    request['configuration']['num_samples'] = 50
    request['configuration']['output_types'] = ['mean']
    return(json.dumps(request))

def getInferences(epname,seriesstr):
    '''Get inferences from the deployed model'''
    runtime = boto3.client('sagemaker-runtime')
    response = runtime.invoke_endpoint(EndpointName=endpoint_name, 
                                   ContentType='application/json', 
                                   Body=build_request(seriesstr))
    return(json.loads(response['Body'].read().decode()))   

def getTime(row):
    if row['TimeHrs'] < 10:
        return str(row['Date2'])[:10] + ' 0' + str(row['TimeHrs']) + ":00:00"
    else:
        return str(row['Date2'])[:10] + ' ' + str(row['TimeHrs']) + ":00:00"

def plotResults(processedDF,seriesstr,result):
    '''Plot the results against the truth'''
    pltFrame = processedDF.replace(to_replace="NaN",value=np.nan,regex=True)
    pltFrame['TimeFull'] = pltFrame.apply(lambda row: getTime(row),axis=1)
    pltFrame['TimeFull'] = pd.to_datetime(pltFrame['TimeFull'])
    pltFrame.set_index('TimeFull',inplace=True)
    predictions = pd.Series(data=result['predictions'][0]['mean'],index=pltFrame.index[pltFrame['Date2'] >= '2008-08-14'][:24])
    pltf = pd.concat([pltFrame,predictions],axis=1)
    pltf[0].plot(kind='line',style='ko--',use_index=True,rot=30,ylim=[0.0,3.0],xlim=[pltf.index[pltf['Date2'] >= '2008-08-13'][0],pltf.index[pltf['Date2'] >= '2008-08-14'][24]],figsize=(20,10))
    pltf[seriesstr].plot(kind='line',style='bo--',use_index=True,rot=30,ylim=[0.0,3.0],xlim=[pltf.index[pltf['Date2'] >= '2008-08-13'][0],pltf.index[pltf['Date2'] >= '2008-08-14'][24]],figsize=(20,10))

## Lets see if we can forecast Global active power and plot the results our model returned

In [ ]:
plotResults(processedDF,
            'Global_active_power',
            getInferences(endpoint_name,'Global_active_power'))

### Lets now see how the same model performs for Global intensity

In [ ]:
plotResults(processedDF,
            'Global_intensity',
            getInferences(endpoint_name,'Global_intensity'))

## Clean Up

In [ ]:
sgmaker.delete_endpoint(EndpointName=endpoint_name)